**Análisis intensivo de pruebas finales'.**

Modelos:
- SVM offline (columna pred_label)
- HT online (columna actividad)
- ARF online (columna actividad_b)

Calcula:
1) Matrices de confusión (PNG).
2) Tiempo de reacción ante cambios de actividad (lag entre cambio en etiqueta y primera predicción correcta).
3) Curvas de aprendizaje (rolling accuracy) para HT/ARF:
4) Estabilidad ("chattering": switches/min de las predicciones).

**IMPORTACIÓN DE LIBRERIAS**

In [1]:
from google.colab import drive
drive.mount('/content/drive')
import argparse
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
import os

Mounted at /content/drive


*Configuraciones*

In [2]:
# ======================= RUTAS =======================
INPUT_CSV = "/content/drive/MyDrive/Colab Notebooks/TESIS/FASE ONLINE/ARCHIVOS ONLINE/windows.csv"
OUT_DIR   = "/content/drive/MyDrive/Colab Notebooks/TESIS/FASE ONLINE/RESULTADOS_SL"

print("CSV existe?", os.path.exists(INPUT_CSV), INPUT_CSV)

BAD_LABELS = {"", " ", "null", "NULL", "None", None}

MODELS = {
    "svm_offline": {"col": "pred_label"},
    "ht_online":   {"col": "actividad"},
    "arf_online":  {"col": "actividad_b"},
}

CSV existe? True /content/drive/MyDrive/Colab Notebooks/TESIS/FASE ONLINE/ARCHIVOS ONLINE/windows.csv


*Utilidades*

In [3]:
def normalize_labels(s: pd.Series) -> pd.Series:
    def _norm(v):
        if pd.isna(v): return None
        sv = str(v).strip()
        return sv if sv not in BAD_LABELS else None
    return s.map(_norm)

def read_csv_robust(path: str) -> pd.DataFrame:
    try:
        return pd.read_csv(path, low_memory=False)
    except UnicodeDecodeError:
        return pd.read_csv(path, low_memory=False, encoding="latin-1")

def parse_time_cols(df: pd.DataFrame) -> pd.DataFrame:
    if "received_at" in df.columns:
        df["received_at"] = pd.to_datetime(df["received_at"], errors="coerce", utc=True)
        df["t"] = df["received_at"]
    else:
        df["t"] = pd.NaT
    if "start_time" in df.columns:
        df["start_time"] = pd.to_datetime(df["start_time"], errors="coerce", utc=True)
    if "end_time" in df.columns:
        df["end_time"] = pd.to_datetime(df["end_time"], errors="coerce", utc=True)
    if df["t"].isna().all() and "end_time" in df.columns:
        df["t"] = df["end_time"]
    if df["t"].isna().all():
        df = df.sort_index().copy()
        base = pd.Timestamp("2000-01-01", tz="UTC")
        df["t"] = [base + pd.Timedelta(seconds=i) for i in range(len(df))]
    return df
def parse_args():
    ap = argparse.ArgumentParser()
    ap.add_argument("--input", required=True, help="Ruta al CSV de windows")
    ap.add_argument("--outdir", default="resultados_windows", help="Carpeta de salida")
    ap.add_argument("--roll-sec", type=int, default=60, help="Ventana (s) para rolling accuracy")
    ap.add_argument("--min-support", type=int, default=5, help="Soporte mínimo por clase para PRF1")
    return ap.parse_args()

def infer_window_seconds(df: pd.DataFrame) -> float:
    if "start_time" in df.columns and "end_time" in df.columns:
        dur = (df["end_time"] - df["start_time"]).dt.total_seconds().dropna()
        if len(dur) > 0:
            return float(np.median(dur.values))
    group_cols = [c for c in ["id_usuario","session_id"] if c in df.columns]
    if not group_cols:
        group_cols = [None]
    gaps = []
    if group_cols == [None]:
        t = df["t"].sort_values()
        dt = t.diff().dt.total_seconds().dropna()
        gaps.extend(dt[dt > 0].tolist())
    else:
        for _, g in df.sort_values("t").groupby(group_cols):
            t = g["t"].sort_values()
            dt = t.diff().dt.total_seconds().dropna()
            gaps.extend(dt[dt > 0].tolist())
    if gaps:
        return float(np.median(gaps))
    return 1.0

def ensure_group_cols(df: pd.DataFrame) -> pd.DataFrame:
    if "id_usuario" not in df.columns:
        df["id_usuario"] = "user_1"
    if "session_id" not in df.columns:
        df["session_id"] = 1
    return df

def prepare(df: pd.DataFrame) -> tuple[pd.DataFrame, float]:
    df = df.copy()
    for col in ["etiqueta","pred_label","actividad","actividad_b"]:
        if col in df.columns:
            df[col] = normalize_labels(df[col])
    df = parse_time_cols(df)
    df = ensure_group_cols(df)
    sort_cols = [c for c in ["id_usuario","session_id","t","id"] if c in df.columns]
    df = df.sort_values(sort_cols, na_position="last").reset_index(drop=True)
    win_s = infer_window_seconds(df)
    return df, win_s

*Funciones para el análisis*

In [4]:
def accuracy_per_group(df: pd.DataFrame, y_col: str, yhat_col: str) -> pd.DataFrame:
    gcols = ["id_usuario","session_id"]
    mask = df[y_col].notna() & df[yhat_col].notna()
    d = df.loc[mask, gcols + [y_col, yhat_col]].copy()
    if d.empty:
        return pd.DataFrame(columns=gcols + ["model","accuracy","n"])
    d["correct"] = (d[y_col] == d[yhat_col]).astype(int)
    res = d.groupby(gcols).agg(accuracy=("correct","mean"), n=("correct","size")).reset_index()
    res["model"] = yhat_col
    return res

def precision_recall_f1(df, y_col, yhat_col, min_support=5) -> pd.DataFrame:
    mask = df[y_col].notna() & df[yhat_col].notna()
    d = df.loc[mask, [y_col, yhat_col]].copy()
    if d.empty:
        return pd.DataFrame(columns=["class","precision","recall","f1","support","model"])
    classes = sorted(set(d[y_col].unique()).union(set(d[yhat_col].unique())))
    rows = []
    for c in classes:
        tp = ((d[y_col]==c) & (d[yhat_col]==c)).sum()
        fp = ((d[y_col]!=c) & (d[yhat_col]==c)).sum()
        fn = ((d[y_col]==c) & (d[yhat_col]!=c)).sum()
        support = (d[y_col]==c).sum()
        prec = tp / (tp + fp) if (tp+fp) > 0 else np.nan
        rec  = tp / (tp + fn) if (tp+fn) > 0 else np.nan
        f1   = (2*prec*rec)/(prec+rec) if (np.isfinite(prec) and np.isfinite(rec) and (prec+rec) > 0) else np.nan
        if support >= min_support:
            rows.append({"class": c, "precision":prec, "recall":rec, "f1":f1, "support":support})
    out = pd.DataFrame(rows)
    out["model"] = yhat_col
    return out

def confusion_matrix(df: pd.DataFrame, y_col: str, yhat_col: str) -> pd.DataFrame:
    mask = df[y_col].notna() & df[yhat_col].notna()
    d = df.loc[mask, [y_col, yhat_col]].copy()
    if d.empty:
        return pd.DataFrame()
    cm = pd.crosstab(index=d[y_col], columns=d[yhat_col], dropna=False)
    cm.index.name = "true"
    cm.columns.name = "pred"
    return cm

def reaction_times(df: pd.DataFrame, y_col: str, yhat_col: str) -> pd.DataFrame:
    gcols = ["id_usuario","session_id"]
    rows = []
    for (u,s), g in df.sort_values("t").groupby(gcols):
        g = g.reset_index(drop=True)
        idx_nonnull = g.index[g[y_col].notna()].tolist()
        if len(idx_nonnull) == 0:
            continue
        last_lbl = None
        for idx in idx_nonnull:
            cur_lbl = g.at[idx, y_col]
            if last_lbl is None:
                last_lbl = cur_lbl
                continue
            if cur_lbl != last_lbl:
                t0 = g.at[idx, "t"]
                sub = g.loc[idx:, [yhat_col, "t"]]
                ok = sub[yhat_col] == cur_lbl
                if ok.any():
                    i_first = ok.idxmax()
                    lag_s = (sub.at[i_first, "t"] - t0).total_seconds()
                    rows.append({"id_usuario":u, "session_id":s, "change_idx":int(idx),
                                 "new_label":cur_lbl, "t_change":t0, "lag_s":lag_s, "model":yhat_col})
                else:
                    rows.append({"id_usuario":u, "session_id":s, "change_idx":int(idx),
                                 "new_label":cur_lbl, "t_change":t0, "lag_s":np.nan, "model":yhat_col})
                last_lbl = cur_lbl
            else:
                last_lbl = cur_lbl
    return pd.DataFrame(rows)

def chatter_rate(df: pd.DataFrame, yhat_col: str) -> pd.DataFrame:
    gcols = ["id_usuario","session_id"]
    rows = []
    for (u,s), g in df.sort_values("t").groupby(gcols):
        gg = g.loc[g[yhat_col].notna(), ["t", yhat_col]].copy()
        if gg.empty or len(gg) < 2:
            continue
        gg = gg.sort_values("t")
        switches = (gg[yhat_col] != gg[yhat_col].shift(1)).sum()
        dt = (gg["t"].iloc[-1] - gg["t"].iloc[0]).total_seconds() / 60.0
        rate = switches / dt if dt > 0 else np.nan
        rows.append({"id_usuario":u, "session_id":s, "switches":int(switches), "minutes":dt, "switches_per_min":rate, "model":yhat_col})
    return pd.DataFrame(rows)

def learning_curve(df: pd.DataFrame, y_col: str, yhat_col: str, roll_sec: int=60) -> pd.DataFrame:
    gcols = ["id_usuario","session_id"]
    rows = []
    for (u,s), g in df.sort_values("t").groupby(gcols):
        g = g.set_index("t")
        m = g[y_col].notna() & g[yhat_col].notna()
        if m.sum() == 0:
            continue
        gg = g.loc[m, [y_col, yhat_col]].copy()
        gg["ok"] = (gg[y_col] == gg[yhat_col]).astype(int)
        rc = gg["ok"].rolling(f"{roll_sec}s").mean()
        tmp = rc.reset_index().rename(columns={"ok":"rolling_acc"})
        tmp["id_usuario"] = u
        tmp["session_id"] = s
        tmp["model"] = yhat_col
        rows.append(tmp)
    if rows:
        return pd.concat(rows, ignore_index=True)
    return pd.DataFrame(columns=["t","rolling_acc","id_usuario","session_id","model"])

*Funciones para graficar*

In [5]:
# --------------------------- Plots ---------------------------
def save_confusion_plot(cm: pd.DataFrame, out_png: Path, title: str):
    if cm.empty: return
    fig, ax = plt.subplots(figsize=(6, 5))
    im = ax.imshow(cm.values, cmap="Blues")
    ax.set_xticks(range(len(cm.columns)))
    ax.set_yticks(range(len(cm.index)))
    ax.set_xticklabels([str(c) for c in cm.columns], rotation=45, ha="right")
    ax.set_yticklabels([str(i) for i in cm.index])
    ax.set_xlabel("Prediction")
    ax.set_ylabel("true (label)")
    ax.set_title(title)
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            ax.text(j, i, int(cm.values[i, j]), ha="center", va="center", color="black")
    cbar = fig.colorbar(im, ax=ax)
    cbar.set_label("Number of windows")
    fig.tight_layout()
    fig.savefig(out_png, dpi=150)
    plt.close(fig)

def save_reaction_boxplot(rt: pd.DataFrame, out_png: Path, title: str):
    # .empty es propiedad, NO se llama como función
    if rt.empty or rt["lag_s"].dropna().empty:
        return
    fig, ax = plt.subplots()
    data = rt["lag_s"].dropna().values
    ax.boxplot(data, vert=True)
    ax.set_ylabel("Lag (s)")
    ax.set_title(title)
    fig.tight_layout()
    fig.savefig(out_png)
    plt.close(fig)


def etiqueta_distribution(df: pd.DataFrame) -> pd.DataFrame:
    d = df.loc[df["etiqueta"].notna(), "etiqueta"].copy()
    if d.empty:
        return pd.DataFrame(columns=["class","count","pct"])
    counts = d.value_counts().rename_axis("class").reset_index(name="count")
    total = counts["count"].sum()
    counts["pct"] = 100.0 * counts["count"] / total
    return counts.sort_values("count", ascending=False).reset_index(drop=True)

def plot_etiqueta_distribution(dist_df: pd.DataFrame, out_png: Path, title: str = "Distribución de etiquetas (ground truth)"):
    if dist_df.empty: return
    fig, ax = plt.subplots(figsize=(8, 4))
    ax.bar(dist_df["class"].astype(str), dist_df["count"])
    ax.set_xlabel("Etiqueta")
    ax.set_ylabel("Número de ventanas etiquetadas")
    ax.set_title(title)
    for i, (cnt, pct) in enumerate(zip(dist_df["count"], dist_df["pct"])):
        ax.text(i, cnt + max(1, 0.01*dist_df["count"].max()), f"{pct:.1f}%", ha="center")
    fig.tight_layout()
    fig.savefig(out_png, dpi=150)
    plt.close(fig)

def aggregate_general_curve(lc_rel: pd.DataFrame, bin_minutes: float = 0.5) -> pd.DataFrame:
    if lc_rel.empty:
        return pd.DataFrame(columns=["rel_min_bin","count","mean","median","q25","q75"])
    rel_bin = (lc_rel["rel_min"] / bin_minutes).floordiv(1) * bin_minutes
    g = lc_rel.assign(rel_min_bin=rel_bin).groupby("rel_min_bin")["rolling_acc"]
    agg = g.agg(count="size", mean="mean", median="median",
                q25=lambda x: x.quantile(0.25), q75=lambda x: x.quantile(0.75)).reset_index()
    return agg.sort_values("rel_min_bin")

def build_learning_curve_relative(df: pd.DataFrame, y_col: str, yhat_col: str, roll_sec: int=60) -> pd.DataFrame:
    gcols = ["id_usuario","session_id"]
    rows = []
    for (u,s), g in df.sort_values("t").groupby(gcols):
        gg = g.set_index("t")
        m = gg[y_col].notna() & gg[yhat_col].notna()
        if m.sum() == 0: continue
        gg = gg.loc[m, [y_col, yhat_col]]
        ok = (gg[y_col] == gg[yhat_col]).astype(int)
        rc = ok.rolling(f"{roll_sec}s").mean().dropna()
        if rc.empty: continue
        t0 = rc.index.min()
        rel_min = (rc.index - t0).total_seconds() / 60.0
        rows.append(pd.DataFrame({"id_usuario": u, "session_id": s, "rel_min": rel_min.values, "rolling_acc": rc.values}))
    if rows: return pd.concat(rows, ignore_index=True)
    return pd.DataFrame(columns=["id_usuario","session_id","rel_min","rolling_acc"])

def plot_session_overlays(lc_rel: pd.DataFrame, out_png: Path, title: str, bin_minutes: float = 0.5, alpha_sessions: float = 0.25, lw_sessions: float = 1.0):
    if lc_rel.empty: return
    agg = aggregate_general_curve(lc_rel, bin_minutes=bin_minutes)
    if agg.empty: return
    fig, ax = plt.subplots(figsize=(8,4))
    for (u, s), g in lc_rel.groupby(["id_usuario","session_id"]):
        g2 = g.sort_values("rel_min")
        ax.plot(g2["rel_min"], g2["rolling_acc"], alpha=alpha_sessions, linewidth=lw_sessions)
    ax.fill_between(agg["rel_min_bin"], agg["q25"], agg["q75"], alpha=0.15, label="IQR (25–75%)")
    ax.plot(agg["rel_min_bin"], agg["mean"], linewidth=2.5, label="Media")
    ax.set_xlabel("Minutos desde inicio")
    ax.set_ylabel("Rolling acc")
    ax.set_title(title)
    ax.set_ylim(0, 1.05)
    ax.grid(True, alpha=0.2)
    ax.legend()
    fig.tight_layout()
    fig.savefig(out_png, dpi=150)
    plt.close(fig)

def plot_general_comparison(agg_ht: pd.DataFrame, agg_arf: pd.DataFrame, out_png: Path, roll_sec: int):
    if agg_ht.empty and agg_arf.empty: return
    fig, axes = plt.subplots(1, 2, figsize=(12,4), sharey=True)
    models = [("HT online", agg_ht), ("ARF online", agg_arf)]
    for ax, (name, agg) in zip(axes, models):
        if agg.empty:
            ax.set_visible(False)
            continue
        ax.fill_between(agg["rel_min_bin"], agg["q25"], agg["q75"], alpha=0.15, label="IQR (25–75%)")
        ax.plot(agg["rel_min_bin"], agg["mean"], linewidth=2.5, label="Media")
        ax.set_xlabel("Minutos desde inicio")
        ax.set_title(f"{name}")
        ax.grid(True, alpha=0.2)
    axes[0].set_ylabel("Rolling acc")
    axes[0].legend(loc="lower right")
    fig.suptitle(f"Curvas de aprendizaje generales ({roll_sec}s)", y=1.02)
    fig.tight_layout()
    fig.savefig(out_png, dpi=150, bbox_inches="tight")
    plt.close(fig)

def build_per_class_pivot(prf_summary: pd.DataFrame) -> pd.DataFrame:
    if prf_summary.empty:
        return pd.DataFrame()

    df = prf_summary.copy()

    # pivot: métricas como columnas por modelo (model_key)
    wide = df.pivot_table(
        index="class",
        columns="model_key",
        values=["precision", "recall", "f1"],
        aggfunc="first"
    )

    # aplanar nombres de columnas: precision_ht_online, f1_arf_online, etc.
    wide.columns = [f"{metric}_{model}" for metric, model in wide.columns]
    wide = wide.reset_index()

    # añadir support (es único por clase)
    supp = (df[["class", "support"]]
            .dropna()
            .drop_duplicates(subset=["class"])
            .set_index("class"))
    wide = wide.set_index("class").join(supp).reset_index()

    # ordenar por support desc (opcional)
    if "support" in wide.columns:
        wide = wide.sort_values("support", ascending=False)

    # redondeo de métricas
    metric_cols = [c for c in wide.columns if any(c.startswith(m) for m in ["precision_", "recall_", "f1_"])]
    wide[metric_cols] = wide[metric_cols].round(3)

    # reordenar columnas
    cols_front = ["class"]
    cols_tail = ["support"] if "support" in wide.columns else []
    other_cols = [c for c in wide.columns if c not in cols_front + cols_tail]
    wide = wide[cols_front + other_cols + cols_tail]

    return wide

def plot_reaction_boxplot_combined(
    reaction_df: pd.DataFrame,
    out_png: Path,
    order=("svm_offline", "ht_online", "arf_online"),
    title="Comparison of reaction lag(s)"
):
    """
    Dibuja un único boxplot con el lag de reacción de todos los modelos.
    - reaction_df: DF que ya generas, con columnas ['lag_s','model_key'].
    - order: orden de modelos a mostrar.
    """
    if reaction_df is None or reaction_df.empty or "lag_s" not in reaction_df.columns:
        return

    data = []
    labels = []
    for key in order:
        sub = reaction_df.loc[reaction_df["model_key"] == key, "lag_s"].dropna()
        if sub.empty:
            continue
        data.append(sub.values)
        labels.append(key.replace("_", " "))

    if not data:
        return

    fig, ax = plt.subplots(figsize=(7,5))
    ax.boxplot(data, labels=labels, showfliers=True)
    ax.set_ylabel("Lag (s)")
    ax.set_title(title)

    # Escala 'symlog' para ver tanto valores pequeños como outliers grandes
    # (lineal cerca de 0, logarítmica en valores altos)
    ax.set_yscale("symlog", linthresh=1)

    fig.tight_layout()
    fig.savefig(out_png, dpi=150)
    plt.close(fig)

def plot_stability_boxplot(
    stability_df: pd.DataFrame,
    out_png: Path,
    order=("svm_offline", "ht_online", "arf_online"),
    title="Comparativa de estabilidad (switches/min)"
):
    """
    Boxplot comparando la estabilidad (switches_per_min) entre modelos.
    - stability_df: DataFrame con columnas ['switches_per_min','model_key'].
    - order: orden en el que mostrar los modelos en el gráfico.
    """
    if stability_df is None or stability_df.empty or "switches_per_min" not in stability_df.columns:
        return

    data = []
    labels = []
    for key in order:
        sub = stability_df.loc[stability_df["model_key"] == key, "switches_per_min"].dropna()
        if sub.empty:
            continue
        data.append(sub.values)
        labels.append(key.replace("_", " "))

    if not data:
        return

    fig, ax = plt.subplots(figsize=(7,5))
    ax.boxplot(data, labels=labels, showfliers=True)
    ax.set_ylabel("Switches per minute")
    ax.set_title(title)
    ax.grid(True, alpha=0.3)

    fig.tight_layout()
    fig.savefig(out_png, dpi=150)
    plt.close(fig)

***MAIN***

In [12]:
def main():
    args = parse_args()
    outdir = Path(args.outdir)
    outdir.mkdir(parents=True, exist_ok=True)

    df = read_csv_robust(args.input)
    df, win_s = prepare(df)

    with open(outdir / "README.txt", "w", encoding="utf-8") as f:
        f.write(f"Archivo analizado: {args.input}\n")
        f.write(f"Filas totales: {len(df)}\n")
        f.write(f"Ventana temporal estimada: {win_s:.3f} s\n")
        f.write(f"Usuarios únicos: {df['id_usuario'].nunique()}\n")
        f.write(f"Sesiones únicas: {df['session_id'].nunique()}\n")

    # --------- Métricas por modelo ---------
    acc_rows, prf_rows = [], []
    for model_key, cfg in MODELS.items():
        col = cfg["col"]
        if col not in df.columns: continue
        acc = accuracy_per_group(df, "etiqueta", col); acc["model_key"] = model_key; acc_rows.append(acc)
        prf = precision_recall_f1(df, "etiqueta", col, min_support=args.min_support); prf["model_key"] = model_key; prf_rows.append(prf)
        cm = confusion_matrix(df, "etiqueta", col)
        save_confusion_plot(cm, outdir / f"confusion_{model_key}.png", f"Confusion matrix ({model_key})")

    metrics_summary = pd.concat(acc_rows, ignore_index=True) if acc_rows else pd.DataFrame()
    prf_summary = pd.concat(prf_rows, ignore_index=True) if prf_rows else pd.DataFrame()

    # Tabla sintetizada por clase (columnas por modelo)
    per_class_pivot = build_per_class_pivot(prf_summary)
    if not per_class_pivot.empty:
        per_class_pivot.to_csv(outdir / "per_class_metrics_pivot.csv", index=False)
        print("\nTabla sintetizada (primeras filas):")
        print(per_class_pivot.head().to_string(index=False))


    # --------- Estabilidad ---------
    stab_rows = []
    for model_key, cfg in MODELS.items():
        col = cfg["col"]
        if col not in df.columns: continue
        stab = chatter_rate(df, col); stab["model_key"] = model_key; stab_rows.append(stab)
    stability = pd.concat(stab_rows, ignore_index=True) if stab_rows else pd.DataFrame()
    stability.to_csv(outdir / "stability_by_session.csv", index=False)

    # Comparativa de estabilidad en un solo gráfico
    plot_stability_boxplot(
        stability_df=stability,
        out_png=outdir / "stability_box_all_models.png",
        order=("svm_offline", "ht_online", "arf_online"),
        title="Stability comparison (switches/min)"
    )

    # --------- Tiempos de reacción ---------
    rt_rows = []
    for model_key, cfg in MODELS.items():
        col = cfg["col"]
        if col not in df.columns: continue
        rt = reaction_times(df, "etiqueta", col)
        rt["model_key"] = model_key
        rt_rows.append(rt)
        # 👉 si no quieres los boxplots separados, comenta esta línea:
        # save_reaction_boxplot(rt, outdir / f"reaction_box_{model_key}.png", f"Lag de reacción ({model_key})")

    reaction = pd.concat(rt_rows, ignore_index=True) if rt_rows else pd.DataFrame()

    plot_reaction_boxplot_combined(
        reaction_df=reaction,
        out_png=outdir / "reaction_box_all_models.png",
        order=("svm_offline", "ht_online", "arf_online"),
        title="Comparison of reaction lag (s)"
    )

    # --------- Curvas de aprendizaje: SOLO overlays + comparativa ---------
    lc_rows = []
    for model_key, cfg in MODELS.items():
        if not model_key.endswith("_online"): continue
        col = cfg["col"]
        if col not in df.columns: continue
        lc = learning_curve(df, "etiqueta", col, roll_sec=args.roll_sec); lc["model_key"] = model_key; lc_rows.append(lc)
    lc_all = pd.concat(lc_rows, ignore_index=True) if lc_rows else pd.DataFrame()

    agg_ht = pd.DataFrame(); agg_arf = pd.DataFrame()

    if "actividad" in df.columns:
        lc_ht_rel = build_learning_curve_relative(df, "etiqueta", "actividad", roll_sec=args.roll_sec)
        if not lc_ht_rel.empty:
            plot_session_overlays(lc_ht_rel, out_png = outdir / "learning_curve_ht_online_overlay.png",
                                  title = f"HT online: sesiones + promedio ({args.roll_sec}s)", bin_minutes = 0.5)
            agg_ht = aggregate_general_curve(lc_ht_rel, bin_minutes=0.5)

    if "actividad_b" in df.columns:
        lc_arf_rel = build_learning_curve_relative(df, "etiqueta", "actividad_b", roll_sec=args.roll_sec)
        if not lc_arf_rel.empty:
            plot_session_overlays(lc_arf_rel, out_png = outdir / "learning_curve_arf_online_overlay.png",
                                  title = f"ARF online: sesiones + promedio ({args.roll_sec}s)", bin_minutes = 0.5)
            agg_arf = aggregate_general_curve(lc_arf_rel, bin_minutes=0.5)

    plot_general_comparison(agg_ht = agg_ht, agg_arf = agg_arf,
                            out_png = outdir / "learning_curve_general_comparison.png",
                            roll_sec = args.roll_sec)


    # --------- Distribución de 'etiqueta' (ground truth) ---------
    dist = etiqueta_distribution(df)
    plot_etiqueta_distribution(dist, out_png = outdir / "etiqueta_distribution.png",
                               title   = "Distribución de etiquetas (ground truth)")

    # --------- Resumen ejecutivo ---------
    summary_rows = []
    for model_key, cfg in MODELS.items():
        col = cfg["col"]
        if not metrics_summary.empty:
            m = metrics_summary.loc[metrics_summary["model_key"]==model_key]
            acc_mean = (m["accuracy"] * m["n"]).sum() / max(1, m["n"].sum()) if not m.empty else np.nan
        else:
            acc_mean = np.nan
        r = reaction.loc[reaction["model_key"]==model_key, "lag_s"].dropna() if not reaction.empty else pd.Series(dtype=float)
        rt_med = float(r.median()) if not r.empty else np.nan
        st = stability.loc[stability["model_key"]==model_key, "switches_per_min"].dropna() if not stability.empty else pd.Series(dtype=float)
        chat_mean = float(st.mean()) if not st.empty else np.nan
        summary_rows.append({"model_key":model_key, "yhat_col":col,
                             "accuracy_mean":acc_mean,
                             "reaction_median_s":rt_med,
                             "switches_per_min_mean":chat_mean,
                             "win_seconds_est":win_s})
    pd.DataFrame(summary_rows).to_csv(outdir / "executive_summary.csv", index=False)

if __name__ == "__main__":
    import sys
    if 'google.colab' in sys.modules and not any(a.startswith("--input") for a in sys.argv):
        sys.argv = [
            "analisis_sl_final.py",
            "--input", INPUT_CSV,
            "--outdir", OUT_DIR,
            "--roll-sec", "60",
            "--min-support", "5",
        ]
    main()



Tabla sintetizada (primeras filas):
   class  f1_arf_online  f1_ht_online  f1_svm_offline  precision_arf_online  precision_ht_online  precision_svm_offline  recall_arf_online  recall_ht_online  recall_svm_offline  support
sentarse          0.795         0.954           0.120                 0.847                0.934                  0.590              0.750             0.974               0.067     4076
 caminar          0.390         0.737           0.643                 0.362                0.774                  0.863              0.423             0.704               0.512     2049
  gradas          0.078         0.297           0.122                 0.064                0.298                  0.561              0.098             0.297               0.068      542
  caerse          0.012         0.439           0.084                 0.021                0.433                  0.044              0.008             0.445               0.931      245


/tmp/ipython-input-1331158214.py:231: MatplotlibDeprecationWarning: The 'labels' parameter of boxplot() has been renamed 'tick_labels' since Matplotlib 3.9; support for the old name will be dropped in 3.11.
  ax.boxplot(data, labels=labels, showfliers=True)
/tmp/ipython-input-1331158214.py:192: MatplotlibDeprecationWarning: The 'labels' parameter of boxplot() has been renamed 'tick_labels' since Matplotlib 3.9; support for the old name will be dropped in 3.11.
  ax.boxplot(data, labels=labels, showfliers=True)
